# Mathematico game

In the notebook, the usage of the package will be demostrated as well as it will offer to play the game interactively.

In [1]:
from mathematico.game import Arena, Board, Mathematico, Player
from mathematico.players import HumanPlayer, RandomPlayer, SimulationPlayer

## Playing a game

To create one mathematico game, you need to instantiate `Mathematico` class, and add a player - in this case a random players.

In [2]:
players = [RandomPlayer(seed=i % 2) for i in range(4)]
game = Mathematico(seed=0)  # with seed, always the same game will be played

for player in players:
    game.add_player(player)

After adding the player, method `Mathematico.play` will simulate one game and return a list of scores for each player in the order they were added. Optional parameter `verbose` controls, how much output will be shown from the game.

In [3]:
results = game.play()
results

[120, 80, 120, 80]

As each player has its own board, to show the final board of the best player, we can use:

In [4]:
best_player_idx = results.index(max(results))
best_player = players[best_player_idx]
print(best_player.board)

+--+--+--+--+--+
| 5|12|13| 9| 1|
+--+--+--+--+--+
| 4| 3| 6|11|11|
+--+--+--+--+--+
| 4|10| 8| 9| 4|
+--+--+--+--+--+
| 2| 2|13| 8| 2|
+--+--+--+--+--+
|10| 9|10| 1|12|
+--+--+--+--+--+


Class `Mathematico` plays only one game, to simulate multiple games, use class `Arena`.

In [5]:
game = Mathematico()
player = SimulationPlayer(max_time=5e4, max_simulations=None)
game.add_player(player)
game.play()

[240]

In [6]:
arena = Arena()
for seed in (0, 5, 42, 43, 69, 420):
    arena.add_player(RandomPlayer(seed))
arena.add_player(SimulationPlayer(max_time=5e4, max_simulations=None))
    
results = arena.run(steps=5, seed=0, verbose=True)

Steps run: 5	Elapsed time: 9.132042646408081


The results is a 2d array with scores for each player.

In [7]:
from statistics import mean

# average random player scores
print(*map(mean, results))

84 96 112 74 78 92 226


## Interactive game

To play game interactively, via command line, use `HumanPlayer`:

In [ ]:
human = HumanPlayer()
game = Mathematico()
game.add_player(human)
game.play()

### Using `ipywidgets`

Alternative is playing using IPython gui elements.

#### Definition of interactive game

In [8]:
import ipywidgets as widgets


class IMathematico: 
    """Enable playing the game by clicking the buttons."""
    
    def set_card_text(self, card_text):
        self.card.value = f"<em style='font-size:20px; padding-left:3em;'>{card_text}</em>"
    
    def create_cell(self, row: int, col: int):
        description = "" if self.board.is_empty(row, col) else self.board.at(row, col)
        btn = widgets.Button(
            description=description,
            disabled=bool(description)
        )
        btn.layout = widgets.Layout(width="80px", height="80px", border="1px solid black")
        btn.game_position = (row, col)
        return btn
    
    def create_board_widget(self):
        items = []
        for row in range(self.board.size):
            for col in range(self.board.size):
                cell = self.create_cell(row, col)
                cell.on_click(self.handle_click)
                items.append(cell)
        return widgets.GridBox(
            items, 
            layout=widgets.Layout(grid_template_columns="repeat(5, auto)")
        )
    
    def __init__(self, seed=None):
        self.game = Mathematico(seed)
        self.board = Board()
        self.current_card = self.game.next_card()
        self.card = widgets.HTML(font_size=15) 
        self.set_card_text(f"Next card: {self.current_card}")
        self.widg = widgets.HBox([
            self.create_board_widget(),
            self.card
        ]) 
        
    def run(self):
        return self.widg
    
    def handle_click(self, x):
        row, col = x.game_position
        x.disabled = True
        x.description = str(self.current_card)
        self.board.make_move((row, col), self.current_card)
        self.current_card = self.game.next_card()
        self.set_card_text(f"Next card: {self.current_card}")
        
        if self.board.occupied_cells == self.board.size ** 2:
            self.set_card_text(f"Game ended, score: {self.board.score()}")

#### Game play

In [9]:
game = IMathematico()
game.run()